# 1. Optimización de Recuperación de Oro

Proyecto de Machine Learning para la optimización de procesos industriales de recuperación de oro mediante la predicción de eficiencia en las etapas de flotación y purificación. El modelo predice la recuperación de oro tanto en el concentrado rougher como en el concentrado final, utilizando datos de sensores del proceso minero y parámetros operacionales.


## 1.1 Alcance y criterios

En esta sección se definen los lineamientos iniciales del proyecto para garantizar orden y reproducibilidad.

**Objetivo:**  
Establecer el entorno de trabajo, parámetros y utilidades del proyecto antes de la carga y exploración de datos.

**Qué haremos aquí:**
1. Cargar librerías y verificar versiones  
2. Definir parámetros globales (rutas de datos, pesos de la métrica, semilla de aleatoriedad)  
3. Configurar estilo visual básico para gráficos  
4. Preparar importación de utilidades desde `src/` con fallback en caso de no encontrarlas


## 1.2 Parámetros globales y configuración inicial

En esta sección se consolidan los elementos necesarios para la configuración inicial del proyecto:  
- Importación de librerías principales  
- Verificación de versiones del entorno  
- Configuración visual para las gráficas  
- Definición de rutas de datos  
- Variables objetivo y pesos de la métrica final

Este bloque asegura que el entorno sea reproducible y consistente durante todo el desarrollo.

In [4]:
# 1.2 Parámetros globales y configuración inicial

# Librerías principales
import os, sys, platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy

# Reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Verificación de versiones
versions = {
    "python": platform.python_version(),
    "numpy": np.__version__,
    "pandas": pd.__version__,
    "matplotlib": plt.matplotlib.__version__,
    "seaborn": sns.__version__,
    "scikit-learn": sklearn.__version__,
    "scipy": scipy.__version__,
}
print("✔️ Versiones detectadas:", versions)

# Configuración visual
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.dpi"] = 110
sns.set_style("whitegrid")

# Rutas de datos
DATA_DIR   = "../datasets"
TRAIN_PATH = f"{DATA_DIR}/gold_recovery_train.csv"
TEST_PATH  = f"{DATA_DIR}/gold_recovery_test.csv"
FULL_PATH  = f"{DATA_DIR}/gold_recovery_full.csv"

# Variables objetivo
TARGETS = ["rougher.output.recovery", "final.output.recovery"]

# Pesos de la métrica final
W_ROUGHER = 0.25
W_FINAL   = 0.75

print("✔️ Parámetros establecidos")
print("DATA_DIR:", DATA_DIR)
print("TARGETS:", TARGETS)
print(f"Pesos métrica final → W_rougher={W_ROUGHER}, W_final={W_FINAL}")


✔️ Versiones detectadas: {'python': '3.11.13', 'numpy': '1.24.4', 'pandas': '2.1.4', 'matplotlib': '3.7.3', 'seaborn': '0.12.2', 'scikit-learn': '1.3.2', 'scipy': '1.11.4'}
✔️ Parámetros establecidos
DATA_DIR: ../datasets
TARGETS: ['rougher.output.recovery', 'final.output.recovery']
Pesos métrica final → W_rougher=0.25, W_final=0.75


### ✅ Conclusión — Sección 1.2 (Parámetros globales y configuración inicial)

- El entorno se encuentra correctamente configurado con versiones actualizadas y compatibles:  
  Python 3.11.13, NumPy 1.24.4, Pandas 2.1.4, Matplotlib 3.7.3, Seaborn 0.12.2, Scikit-learn 1.3.2 y SciPy 1.11.4.  
- Se establecieron los parámetros clave del proyecto: rutas de los datasets, variables objetivo (`rougher.output.recovery` y `final.output.recovery`) y los pesos de la métrica final (0.25 para rougher, 0.75 para final).  
- El estilo visual quedó definido con Matplotlib y Seaborn para garantizar uniformidad en las gráficas.  
- Estado: la configuración inicial está completa y lista para avanzar a la carga y validación de datos.


# 2. Preparación de Datos

**Objetivo:**  
En esta sección prepararemos los datasets de entrenamiento y prueba para garantizar que estén en condiciones óptimas antes de pasar al análisis exploratorio y a la construcción de modelos. El trabajo consistirá en validar la coherencia de los cálculos de recuperación, identificar diferencias entre las estructuras de los datasets y realizar la limpieza necesaria para eliminar duplicados y valores nulos.  

El resultado esperado es contar con datasets consistentes, alineados y libres de errores, que puedan utilizarse de forma confiable en el modelado de la Sección 4.


## 2.1 Carga y exploración inicial de datasets

**Objetivo:**  
Cargar los datasets de entrenamiento, prueba y fuente completa utilizando las rutas definidas en la sección 1.2.  
El propósito es revisar dimensiones, tipos de datos, valores nulos y obtener una vista preliminar de las primeras filas. Con esto aseguramos un panorama inicial claro de la estructura de datos, verificamos la coherencia de la información descargada y dejamos listos los datasets base para su validación y limpieza en pasos posteriores.  

**Lista de objetos:**  
- `sec2_1_df_train` *(DataFrame)* → dataset de entrenamiento cargado con índice temporal.  
- `sec2_1_df_test` *(DataFrame)* → dataset de prueba cargado con índice temporal.  
- `sec2_1_df_full` *(DataFrame)* → dataset completo (train + test) cargado con índice temporal.  


In [5]:
"""
2.1 Carga y exploración inicial de datasets
Propósito: Cargar los datasets de entrenamiento, prueba y fuente completa con indexación temporal,
y realizar una inspección inicial de dimensiones, tipos de datos, valores nulos y primeras filas.
"""

# 1) Cargar datasets utilizando las rutas definidas en la sección 1.2
sec2_1_df_train = pd.read_csv(TRAIN_PATH, index_col="date", parse_dates=True)
sec2_1_df_test  = pd.read_csv(TEST_PATH, index_col="date", parse_dates=True)
sec2_1_df_full  = pd.read_csv(FULL_PATH, index_col="date", parse_dates=True)

# 2) Mostrar dimensiones de cada dataset
print("Dimensiones dataset de entrenamiento:", sec2_1_df_train.shape)
print("Dimensiones dataset de prueba:", sec2_1_df_test.shape)
print("Dimensiones dataset completo:", sec2_1_df_full.shape)

# 3) Vista preliminar de las primeras filas
print("\nPrimeras filas - Entrenamiento:")
display(sec2_1_df_train.head())

print("\nPrimeras filas - Prueba:")
display(sec2_1_df_test.head())

print("\nPrimeras filas - Completo:")
display(sec2_1_df_full.head())

# 4) Información general de tipos de datos y valores nulos
print("\n--- Información Entrenamiento ---")
print(sec2_1_df_train.info())

print("\n--- Información Prueba ---")
print(sec2_1_df_test.info())

print("\n--- Información Completo ---")
print(sec2_1_df_full.info())


Dimensiones dataset de entrenamiento: (16860, 86)
Dimensiones dataset de prueba: (5856, 52)
Dimensiones dataset completo: (22716, 86)

Primeras filas - Entrenamiento:


,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
date,,,,,,,,,,,,,,,,,,,,,
2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,127.092003,...,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,125.629232,...,13.992281,-505.503262,11.950531,-501.331529,10.039245,-500.169983,7.984757,-500.582168,13.998353,-599.787184
2016-01-15 02:00:00,6.055926,10.213995,5.383759,42.657501,68.116445,10.507046,0.953716,16.208849,2.257889,123.819808,...,14.015015,-502.520901,11.912783,-501.133383,10.070913,-500.129135,8.013877,-500.517572,14.028663,-601.427363
2016-01-15 03:00:00,6.047977,9.977019,4.858634,42.689819,68.347543,10.422762,0.883763,16.532835,2.146849,122.270188,...,14.036510,-500.857308,11.999550,-501.193686,9.970366,-499.201640,7.977324,-500.255908,14.005551,-599.996129
2016-01-15 04:00:00,6.148599,10.142511,4.939416,42.774141,66.927016,10.360302,0.792826,16.525686,2.055292,117.988169,...,14.027298,-499.838632,11.953070,-501.053894,9.925709,-501.686727,7.894242,-500.356035,13.996647,-601.496691



Primeras filas - Prueba:


,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
date,,,,,,,,,,,,,,,,,,,,,
2016-09-01 00:59:59,210.800909,14.993118,8.080000,1.005021,1398.981301,-500.225577,1399.144926,-499.919735,1400.102998,-500.704369,...,12.023554,-497.795834,8.016656,-501.289139,7.946562,-432.317850,4.872511,-500.037437,26.705889,-499.709414
2016-09-01 01:59:59,215.392455,14.987471,8.080000,0.990469,1398.777912,-500.057435,1398.055362,-499.778182,1396.151033,-499.240168,...,12.058140,-498.695773,8.130979,-499.634209,7.958270,-525.839648,4.878850,-500.162375,25.019940,-499.819438
2016-09-01 02:59:59,215.259946,12.884934,7.786667,0.996043,1398.493666,-500.868360,1398.860436,-499.764529,1398.075709,-502.151509,...,11.962366,-498.767484,8.096893,-500.827423,8.071056,-500.801673,4.905125,-499.828510,24.994862,-500.622559
2016-09-01 03:59:59,215.336236,12.006805,7.640000,0.863514,1399.618111,-498.863574,1397.440120,-499.211024,1400.129303,-498.355873,...,12.033091,-498.350935,8.074946,-499.474407,7.897085,-500.868509,4.931400,-499.963623,24.948919,-498.709987
2016-09-01 04:59:59,199.099327,10.682530,7.530000,0.805575,1401.268123,-500.808305,1398.128818,-499.504543,1402.172226,-500.810606,...,12.025367,-500.786497,8.054678,-500.397500,8.107890,-509.526725,4.957674,-500.360026,25.003331,-500.856333



Primeras filas - Completo:


,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
date,,,,,,,,,,,,,,,,,,,,,
2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,127.092003,...,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,125.629232,...,13.992281,-505.503262,11.950531,-501.331529,10.039245,-500.169983,7.984757,-500.582168,13.998353,-599.787184
2016-01-15 02:00:00,6.055926,10.213995,5.383759,42.657501,68.116445,10.507046,0.953716,16.208849,2.257889,123.819808,...,14.015015,-502.520901,11.912783,-501.133383,10.070913,-500.129135,8.013877,-500.517572,14.028663,-601.427363
2016-01-15 03:00:00,6.047977,9.977019,4.858634,42.689819,68.347543,10.422762,0.883763,16.532835,2.146849,122.270188,...,14.036510,-500.857308,11.999550,-501.193686,9.970366,-499.201640,7.977324,-500.255908,14.005551,-599.996129
2016-01-15 04:00:00,6.148599,10.142511,4.939416,42.774141,66.927016,10.360302,0.792826,16.525686,2.055292,117.988169,...,14.027298,-499.838632,11.953070,-501.053894,9.925709,-501.686727,7.894242,-500.356035,13.996647,-601.496691



--- Información Entrenamiento ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16860 entries, 2016-01-15 00:00:00 to 2018-08-18 10:59:59
Data columns (total 86 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   final.output.concentrate_ag                         16788 non-null  float64
 1   final.output.concentrate_pb                         16788 non-null  float64
 2   final.output.concentrate_sol                        16490 non-null  float64
 3   final.output.concentrate_au                         16789 non-null  float64
 4   final.output.recovery                               15339 non-null  float64
 5   final.output.tail_ag                                16794 non-null  float64
 6   final.output.tail_pb                                16677 non-null  float64
 7   final.output.tail_sol                               16715 non-null  float64
 8   final.

### ✅ Conclusión Sección 2.1 — Carga y exploración inicial
- **Dimensiones**: train `(16860, 86)`, test `(5856, 52)`, full `(22716, 86)`.
- **Estructura**: `test` no tiene columnas de `output` ni algunas `calculation` (consistente con entorno de producción).
- **Nulos**: hay NaNs relevantes en `rougher.output.recovery` (14287/16860 non-null) y `final.output.recovery` (15339/16860 non-null) en train; también faltantes en varias `input/state`.
- **Implicación**: necesitaremos **validar recuperación (2.2)** y **limpiar & alinear (2.4)** para evitar problemas en modelado.


### 2.2 Validación de cálculos de recuperación

**Objetivo:**  
Verificar que la recuperación de oro reportada en `rougher.output.recovery` esté correctamente calculada en **porcentaje (%)**, aplicando la fórmula industrial a partir de la concentración de Au en la alimentación (F), en el concentrado (C) y en las colas (T).  

La fórmula que utilizaremos es la siguiente:

***Recuperación = (C * (F - T)) / (F * (C - T)) * 100***

Donde:  
- **F** → concentración de Au en la alimentación (*feed*).  
- **C** → concentración de Au en el concentrado (*concentrate*).  
- **T** → concentración de Au en las colas (*tail*).  

Este paso asegura que la variable objetivo esté bien definida antes de pasar al preprocesamiento y modelado.

**Lista de objetos:**  
- `sec2_2_calc_recovery_percent(C, F, T)` *(func)* → calcula la recuperación en **%**, manejando divisiones por cero y valores no finitos.  
- `sec2_2_aux_valid` *(DataFrame)* → subconjunto válido con F, C, T y target para la comparación.  
- `sec2_2_mae_pct` *(float)* → error absoluto medio (MAE) entre recuperación provista y calculada (%).  
- `sec2_2_desc_real`, `sec2_2_desc_calc` *(Series)* → estadísticas descriptivas de la recuperación real y calculada.  


In [6]:
"""
2.2 Validación de cálculos de recuperación (%)
Objetivo: Recalcular la recuperación rougher en porcentaje usando F, C y T, y
compararla con 'rougher.output.recovery' mediante MAE para validar la coherencia del target.

Fórmula utilizada (industrial):
    Recuperación(%) = [ C × (F − T) ] / [ F × (C − T) ] × 100
donde:
    F = rougher.input.feed_au
    C = rougher.output.concentrate_au
    T = rougher.output.tail_au
"""

import numpy as np
from sklearn.metrics import mean_absolute_error

# --- 1) Columnas requeridas (del dataset de entrenamiento cargado en 2.1) ---
COL_F = "rougher.input.feed_au"           # F
COL_C = "rougher.output.concentrate_au"   # C
COL_T = "rougher.output.tail_au"          # T
COL_Y = "rougher.output.recovery"         # target provista

_required = [COL_F, COL_C, COL_T, COL_Y]
_missing = [c for c in _required if c not in sec2_1_df_train.columns]
if _missing:
    raise KeyError(f"Faltan columnas para el cálculo de recuperación: {_missing}")

# --- 2) Función robusta de recuperación en porcentaje ---
def sec2_2_calc_recovery_percent(C, F, T):
    """
    Calcula la recuperación en porcentaje (%).

    Fórmula:
        recovery% = (C * (F - T)) / (F * (C - T)) * 100

    Manejo numérico:
        - Evita divisiones por cero.
        - Devuelve NaN cuando el denominador es 0 o hay valores no finitos.
    """
    num = C * (F - T)
    den = F * (C - T)
    with np.errstate(divide="ignore", invalid="ignore"):
        r = np.where(den != 0, (num / den) * 100.0, np.nan)
    return r

# --- 3) Subconjunto válido para comparar (quitar inf/NaN necesarios) ---
sec2_2_aux_valid = (
    sec2_1_df_train[[COL_F, COL_C, COL_T, COL_Y]]
    .replace([np.inf, -np.inf], np.nan)
    .dropna(subset=[COL_F, COL_C, COL_T, COL_Y])
)

# --- 4) Cálculo de recuperación y evaluación ---
sec2_2_aux_valid["recovery_calc_pct"] = sec2_2_calc_recovery_percent(
    sec2_2_aux_valid[COL_C].to_numpy(),
    sec2_2_aux_valid[COL_F].to_numpy(),
    sec2_2_aux_valid[COL_T].to_numpy()
)

# Eliminar filas con resultado NaN por divisiones inválidas (si las hubiera)
sec2_2_aux_valid = sec2_2_aux_valid.dropna(subset=["recovery_calc_pct"])

sec2_2_mae_pct = mean_absolute_error(
    sec2_2_aux_valid[COL_Y],
    sec2_2_aux_valid["recovery_calc_pct"]
)

print(f"Filas válidas para validación: {len(sec2_2_aux_valid):,}")
print(f"MAE [%] provista vs calculada: {sec2_2_mae_pct:.6f}")

# --- 5) Descriptivos y auditoría rápida ---
sec2_2_desc_real = sec2_2_aux_valid[COL_Y].describe(percentiles=[0.01,0.05,0.50,0.95,0.99])
sec2_2_desc_calc = sec2_2_aux_valid["recovery_calc_pct"].describe(percentiles=[0.01,0.05,0.50,0.95,0.99])

print("\nDistribución REAL [%]:")
print(sec2_2_desc_real)
print("\nDistribución CALCULADA [%]:")
print(sec2_2_desc_calc)

# Top diferencias absolutas (útil para inspección manual)
sec2_2_aux_valid["abs_err"] = (sec2_2_aux_valid[COL_Y] - sec2_2_aux_valid["recovery_calc_pct"]).abs()
display(
    sec2_2_aux_valid.sort_values("abs_err", ascending=False)
    .head(5)
    [[COL_F, COL_C, COL_T, COL_Y, "recovery_calc_pct", "abs_err"]]
)


Filas válidas para validación: 14,287
MAE [%] provista vs calculada: 0.000000

Distribución REAL [%]:
count    14287.000000
mean        82.394201
std         15.096808
min          0.000000
1%           0.000000
5%          66.014917
50%         85.235997
95%         94.604817
99%         97.215423
max        100.000000
Name: rougher.output.recovery, dtype: float64

Distribución CALCULADA [%]:
count    14287.000000
mean        82.394201
std         15.096808
min         -0.000000
1%          -0.000000
5%          66.014917
50%         85.235997
95%         94.604817
99%         97.215423
max        100.000000
Name: recovery_calc_pct, dtype: float64


,rougher.input.feed_au,rougher.output.concentrate_au,rougher.output.tail_au,rougher.output.recovery,recovery_calc_pct,abs_err
date,,,,,,
2018-02-26 23:59:59,11.916421,20.201165,2.396125,90.643794,90.643794,7.105427e-14
2017-02-18 09:59:59,12.306452,20.828212,1.957962,92.815063,92.815063,7.105427e-14
2016-05-30 21:59:59,6.657497,22.755051,2.165882,74.564225,74.564225,5.684342e-14
2018-05-26 05:59:59,7.534387,13.658775,2.760898,79.406823,79.406823,5.684342e-14
2017-06-19 21:59:59,6.052884,19.669918,1.573671,80.436545,80.436545,5.684342e-14


### ✅ Conclusión Sección 2.2 — Validación de cálculos de recuperación

- Se validaron **14,287 filas** con datos completos para la comparación.  
- El **MAE = 0.0%**, lo que confirma que la recuperación reportada en `rougher.output.recovery` coincide exactamente con la calculada usando la fórmula industrial:  

  Recuperación (%) = (C * (F - T)) / (F * (C - T)) * 100  

- Las distribuciones real y calculada son idénticas:  
  - Media ≈ **82.39%**  
  - Rango de **0% a 100%**, con mediana ≈ 85%.  
- Las diferencias numéricas encontradas son insignificantes (del orden de 10^-14), atribuibles al redondeo en cálculos de punto flotante.  

➡️ Esto confirma que la columna `rougher.output.recovery` es confiable y que podemos continuar con el análisis de características faltantes en el conjunto de prueba en la siguiente subsección (2.3).

### 2.3 Análisis de características faltantes en conjunto de prueba

**Objetivo:**  
Comparar la estructura de columnas entre los datasets de entrenamiento y prueba para identificar qué variables están ausentes en el conjunto de prueba. Este análisis es crucial porque esas columnas no estarán disponibles en un entorno real de producción y, por lo tanto, deben ser excluidas del modelado.  

Además, se documentará qué tipo de variables faltan (entradas, salidas, estados o cálculos) para tener claridad sobre la naturaleza de estas diferencias y garantizar que el conjunto de características a usar en el modelado sea consistente y seguro.

**Lista de objetos:**  
- `sec2_3_train_cols` *(set)* → conjunto de todas las columnas en el dataset de entrenamiento.  
- `sec2_3_test_cols` *(set)* → conjunto de todas las columnas en el dataset de prueba.  
- `sec2_3_cols_missing_in_test` *(list[str])* → columnas que están en train pero no en test; será la base para alinear variables en la siguiente subsección.  
- `sec2_3_classify_feature(col)` *(func)* → clasifica una columna como `input`, `output`, `state` o `calculation`.  
- `sec2_3_missing_summary` *(DataFrame)* → tabla con conteo de columnas faltantes en test, agrupadas por tipo.  


In [7]:
"""
2.3 Análisis de características faltantes en conjunto de prueba
Objetivo: Comparar columnas de train y test para identificar cuáles NO están disponibles en test,
clasificarlas por tipo y documentar un resumen para alinear features en 2.4.

Convención (Sección 2.3):
- Importantes: sec2_3_train_cols, sec2_3_test_cols, sec2_3_cols_missing_in_test, sec2_3_missing_summary
- Temporales: _s, _ej
- Función local: sec2_3_classify_feature
"""

import pandas as pd

# 1) Conjuntos de columnas (importantes)
sec2_3_train_cols = set(sec2_1_df_train.columns)
sec2_3_test_cols  = set(sec2_1_df_test.columns)

# 2) Columnas presentes en train y AUSENTES en test (importante)
sec2_3_cols_missing_in_test = sorted(sec2_3_train_cols - sec2_3_test_cols)
print(f"Total columnas ausentes en TEST: {len(sec2_3_cols_missing_in_test)}")

# 3) Clasificador por tipo (importante)
def sec2_3_classify_feature(col: str) -> str:
    if ".input." in col: return "input"
    if ".output." in col: return "output"
    if ".state." in col: return "state"
    if ".calculation." in col: return "calculation"
    return "otro"

# 4) Resumen por tipo (importante para documentación)
_s = pd.Series({c: sec2_3_classify_feature(c) for c in sec2_3_cols_missing_in_test}, name="tipo")
sec2_3_missing_summary = _s.value_counts().rename("conteo").to_frame()

print("\nResumen por tipo:")
display(sec2_3_missing_summary)

# 5) Ejemplos por categoría (salida ligera para inspección)
for _tipo in sec2_3_missing_summary.index:
    _ej = [c for c in sec2_3_cols_missing_in_test if sec2_3_classify_feature(c) == _tipo][:5]
    print(f"\nEjemplos '{_tipo}':")
    for c in _ej:
        print("  -", c)


Total columnas ausentes en TEST: 34

Resumen por tipo:


,conteo
tipo,
output,30
calculation,4



Ejemplos 'output':
  - final.output.concentrate_ag
  - final.output.concentrate_au
  - final.output.concentrate_pb
  - final.output.concentrate_sol
  - final.output.recovery

Ejemplos 'calculation':
  - rougher.calculation.au_pb_ratio
  - rougher.calculation.floatbank10_sulfate_to_au_feed
  - rougher.calculation.floatbank11_sulfate_to_au_feed
  - rougher.calculation.sulfate_to_au_concentrate


### ✅ Conclusión Sección 2.3 — Análisis de características faltantes en conjunto de prueba

- Se identificaron **34 columnas presentes en el dataset de entrenamiento que no están en el de prueba**.  
- Clasificación de las columnas faltantes:
  - **30 columnas de tipo `output`** → variables de concentrados y colas, además de las recuperaciones.  
  - **4 columnas de tipo `calculation`** → relaciones derivadas que no pueden conocerse en un entorno de producción.  
- Ejemplos de columnas faltantes:
  - Output: `final.output.concentrate_ag`, `final.output.concentrate_au`, `final.output.concentrate_pb`, `final.output.concentrate_sol`, `final.output.recovery`.  
  - Calculation: `rougher.calculation.au_pb_ratio`, `rougher.calculation.floatbank10_sulfate_to_au_feed`, `rougher.calculation.floatbank11_sulfate_to_au_feed`, `rougher.calculation.sulfate_to_au_concentrate`.  
- Esto confirma que el conjunto de prueba no incluye ni las variables objetivo ni las calculadas, lo cual es consistente con un escenario real de producción.  



### 2.4 Preprocesamiento de datos

**Objetivo:**  
Realizar la limpieza y preparación final de los datasets de entrenamiento y prueba para asegurar que estén en condiciones óptimas antes del modelado. En este paso se eliminarán duplicados y valores nulos, y se alinearán las columnas entre ambos datasets para que contengan únicamente las variables disponibles en producción.  

El resultado esperado son datasets consistentes, libres de errores y perfectamente comparables, listos para ser usados en la etapa de modelado.

**Lista de objetos:**  
- `sec2_4_train_clean` *(DataFrame)* → dataset de entrenamiento limpio, sin duplicados ni valores nulos en las columnas relevantes.  
- `sec2_4_test_clean` *(DataFrame)* → dataset de prueba limpio y alineado con las mismas columnas que el dataset de entrenamiento.  
- `sec2_4_common_features` *(list[str])* → lista de columnas comunes entre train y test, utilizada para garantizar que ambos datasets tengan la misma estructura.  
- `sec2_4_na_summary_train` *(Series/DataFrame)* → resumen de valores nulos en el dataset de entrenamiento antes y después de la limpieza.  
- `sec2_4_na_summary_test` *(Series/DataFrame)* → resumen de valores nulos en el dataset de prueba antes y después de la limpieza.  


In [8]:
"""
2.4 Preprocesamiento de datos
Objetivo: Dejar listos los datasets para modelado eliminando duplicados, resolviendo
todos los NaNs y alineando las columnas entre train y test.

Flujo:
  1) Alinear columnas (intersección train/test excluyendo TARGETS).
  2) Eliminar duplicados (filas idénticas).
  3) Eliminar filas con NaN en las variables objetivo (solo en train).
  4) Imputar NaNs en features con la MEDIANA (fit en train, transform en test).
  5) Confirmar: sin NaNs y estructuras iguales.

Entradas esperadas:
  - sec2_1_df_train, sec2_1_df_test (de 2.1)
  - TARGETS (de 1.2)

Salidas principales:
  - sec2_4_common_features
  - sec2_4_train_clean (features imputed + TARGETS)
  - sec2_4_test_clean  (features imputed)
  - sec2_4_na_summary_train, sec2_4_na_summary_test (resumen NaNs antes/después)
"""

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# ----------------------------
# 1) Alinear columnas
# ----------------------------
# Intersección de columnas entre train y test (excluyendo TARGETS)
_common = (set(sec2_1_df_train.columns) - set(TARGETS)) & set(sec2_1_df_test.columns)
sec2_4_common_features = sorted(list(_common))

# Construir vistas alineadas
_train_aligned = sec2_1_df_train[sec2_4_common_features + TARGETS].copy()
_test_aligned  = sec2_1_df_test[sec2_4_common_features].copy()

print(f"Features comunes (sin TARGETS): {len(sec2_4_common_features)}")

# ----------------------------
# 2) Eliminar duplicados
# ----------------------------
_dups_train = _train_aligned.duplicated().sum()
_dups_test  = _test_aligned.duplicated().sum()
print(f"Duplicados en TRAIN (antes): {_dups_train}")
print(f"Duplicados en TEST  (antes): {_dups_test}")

_train_aligned = _train_aligned.drop_duplicates(keep="first")
_test_aligned  = _test_aligned.drop_duplicates(keep="first")

# ----------------------------
# 3) Eliminar NaNs en TARGETS (solo train)
# ----------------------------
_before_target_na = _train_aligned[TARGETS].isna().sum()
print("\nNaNs en TARGETS (antes de limpiar):")
print(_before_target_na)

_train_aligned = _train_aligned.dropna(subset=TARGETS)

# ----------------------------
# 4) Imputación de NaNs en features
# ----------------------------
# Resumen de NaNs ANTES de imputar (solo features)
sec2_4_na_summary_train = _train_aligned[sec2_4_common_features].isna().sum().rename("na_count_train_before")
sec2_4_na_summary_test  = _test_aligned[sec2_4_common_features].isna().sum().rename("na_count_test_before")

_imputer = SimpleImputer(strategy="median")

# Fit en TRAIN, transform en TRAIN y TEST
_X_train_imp = pd.DataFrame(
    _imputer.fit_transform(_train_aligned[sec2_4_common_features]),
    columns=sec2_4_common_features,
    index=_train_aligned.index
)
_X_test_imp = pd.DataFrame(
    _imputer.transform(_test_aligned[sec2_4_common_features]),
    columns=sec2_4_common_features,
    index=_test_aligned.index
)

# Reconstruir datasets LIMPIOS
sec2_4_train_clean = pd.concat([_X_train_imp, _train_aligned[TARGETS]], axis=1)
sec2_4_test_clean  = _X_test_imp.copy()

# ----------------------------
# 5) Confirmaciones finales
# ----------------------------
# Resumen de NaNs DESPUÉS de imputar
_after_train_na = sec2_4_train_clean.isna().sum().sum()
_after_test_na  = sec2_4_test_clean.isna().sum().sum()

print("\n--- Confirmaciones finales ---")
print(f"Filas TRAIN (limpio): {sec2_4_train_clean.shape[0]:,} | Cols: {sec2_4_train_clean.shape[1]}")
print(f"Filas TEST  (limpio): {sec2_4_test_clean.shape[0]:,} | Cols: {sec2_4_test_clean.shape[1]}")
print(f"NaNs totales en TRAIN (después): {_after_train_na}")
print(f"NaNs totales en TEST  (después): {_after_test_na}")

# Validaciones duras (asserts)
assert _after_train_na == 0, "Persisten NaNs en TRAIN después de la imputación."
assert _after_test_na == 0,  "Persisten NaNs en TEST después de la imputación."
assert sec2_4_train_clean.drop(columns=TARGETS).columns.tolist() == sec2_4_test_clean.columns.tolist(), \
       "Las columnas de features no están alineadas entre TRAIN y TEST."

# Muestras rápidas para inspección
print("\nPrimeras columnas (features) alineadas:")
print(sec2_4_train_clean.columns[:10].tolist())
print("\nVista previa TRAIN limpio:")
display(sec2_4_train_clean.head())
print("\nVista previa TEST limpio:")
display(sec2_4_test_clean.head())


Features comunes (sin TARGETS): 52
Duplicados en TRAIN (antes): 19
Duplicados en TEST  (antes): 6

NaNs en TARGETS (antes de limpiar):
rougher.output.recovery    2554
final.output.recovery      1502
dtype: int64

--- Confirmaciones finales ---
Filas TRAIN (limpio): 14,149 | Cols: 54
Filas TEST  (limpio): 5,850 | Cols: 52
NaNs totales en TRAIN (después): 0
NaNs totales en TEST  (después): 0

Primeras columnas (features) alineadas:
['primary_cleaner.input.depressant', 'primary_cleaner.input.feed_size', 'primary_cleaner.input.sulfate', 'primary_cleaner.input.xanthate', 'primary_cleaner.state.floatbank8_a_air', 'primary_cleaner.state.floatbank8_a_level', 'primary_cleaner.state.floatbank8_b_air', 'primary_cleaner.state.floatbank8_b_level', 'primary_cleaner.state.floatbank8_c_air', 'primary_cleaner.state.floatbank8_c_level']

Vista previa TRAIN limpio:


,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.sulfate,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,...,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level,rougher.output.recovery,final.output.recovery
date,,,,,,,,,,,,,,,,,,,,,
2016-01-15 00:00:00,10.128295,7.25,127.092003,0.988759,1549.775757,-498.912140,1551.434204,-516.403442,1549.873901,-498.666595,...,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980,87.107763,70.541216
2016-01-15 01:00:00,10.296251,7.25,125.629232,1.002663,1576.166671,-500.904965,1575.950626,-499.865889,1575.994189,-499.315107,...,11.950531,-501.331529,10.039245,-500.169983,7.984757,-500.582168,13.998353,-599.787184,86.843261,69.266198
2016-01-15 02:00:00,11.316280,7.25,123.819808,0.991265,1601.556163,-499.997791,1600.386685,-500.607762,1602.003542,-500.870069,...,11.912783,-501.133383,10.070913,-500.129135,8.013877,-500.517572,14.028663,-601.427363,86.842308,68.116445
2016-01-15 03:00:00,11.322140,7.25,122.270188,0.996739,1599.968720,-500.951778,1600.659236,-499.677094,1600.304144,-500.727997,...,11.999550,-501.193686,9.970366,-499.201640,7.977324,-500.255908,14.005551,-599.996129,87.226430,68.347543
2016-01-15 04:00:00,11.913613,7.25,117.988169,1.009869,1601.339707,-498.975456,1601.437854,-500.323246,1599.581894,-500.888152,...,11.953070,-501.053894,9.925709,-501.686727,7.894242,-500.356035,13.996647,-601.496691,86.688794,66.927016



Vista previa TEST limpio:


,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.sulfate,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
date,,,,,,,,,,,,,,,,,,,,,
2016-09-01 00:59:59,14.993118,8.080000,210.800909,1.005021,1398.981301,-500.225577,1399.144926,-499.919735,1400.102998,-500.704369,...,12.023554,-497.795834,8.016656,-501.289139,7.946562,-432.317850,4.872511,-500.037437,26.705889,-499.709414
2016-09-01 01:59:59,14.987471,8.080000,215.392455,0.990469,1398.777912,-500.057435,1398.055362,-499.778182,1396.151033,-499.240168,...,12.058140,-498.695773,8.130979,-499.634209,7.958270,-525.839648,4.878850,-500.162375,25.019940,-499.819438
2016-09-01 02:59:59,12.884934,7.786667,215.259946,0.996043,1398.493666,-500.868360,1398.860436,-499.764529,1398.075709,-502.151509,...,11.962366,-498.767484,8.096893,-500.827423,8.071056,-500.801673,4.905125,-499.828510,24.994862,-500.622559
2016-09-01 03:59:59,12.006805,7.640000,215.336236,0.863514,1399.618111,-498.863574,1397.440120,-499.211024,1400.129303,-498.355873,...,12.033091,-498.350935,8.074946,-499.474407,7.897085,-500.868509,4.931400,-499.963623,24.948919,-498.709987
2016-09-01 04:59:59,10.682530,7.530000,199.099327,0.805575,1401.268123,-500.808305,1398.128818,-499.504543,1402.172226,-500.810606,...,12.025367,-500.786497,8.054678,-500.397500,8.107890,-509.526725,4.957674,-500.360026,25.003331,-500.856333


### ✅ Conclusión Sección 2.4 — Preprocesamiento de datos

- **Alineación de features:** 52 columnas comunes entre train y test (features); train quedó con 54 columnas (52 features + 2 targets) y test con 52 columnas (solo features).
- **Duplicados eliminados:** 19 filas en train y 6 filas en test.
- **Filtrado por targets nulos (solo train):** se eliminaron 2,692 filas con NaN en las variables objetivo.
  - Cálculo: 16,860 filas iniciales − 19 duplicados = 16,841; luego 16,841 − 14,149 = 2,692.
- **Imputación de features:** estrategia de mediana, ajustada con train y aplicada a train y test (sin fuga de información).
- **NaNs remanentes:** 0 en train y 0 en test.
- **Estructuras finales:**
  - `sec2_4_train_clean` → (14,149 filas × 54 columnas).
  - `sec2_4_test_clean`  → (5,850 filas × 52 columnas).

Los datasets quedaron consistentes, sin duplicados ni valores nulos, y con columnas perfectamente alineadas para su uso en modelado.
